<a href="https://colab.research.google.com/github/sunlaito/LearnPennyLane/blob/main/Application1_VQE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ref: https://pennylane.ai/qml/demos/tutorial_vqe.html

In [ ]:
!pip install pennylane --upgrade

In [ ]:
!pip install pennylane-qchem # Restart Runtime in Colab after installation! 

In [2]:
qml.about()

Name: PennyLane
Version: 0.13.0
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/XanaduAI/pennylane
Author: None
Author-email: None
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: scipy, autograd, networkx, toml, appdirs, numpy, semantic-version
Required-by: PennyLane-Qchem
Platform info:           Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
Python version:          3.6.9
Numpy version:           1.19.5
Scipy version:           1.4.1
Installed devices:
- default.gaussian (PennyLane-0.13.0)
- default.mixed (PennyLane-0.13.0)
- default.qubit (PennyLane-0.13.0)
- default.qubit.autograd (PennyLane-0.13.0)
- default.qubit.tf (PennyLane-0.13.0)
- default.tensor (PennyLane-0.13.0)
- default.tensor.tf (PennyLane-0.13.0)


In [3]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

In [ ]:
# upload the geometry file "h2.xyz"
# h2.xyz can be downloaded via this link https://pennylane.ai/qml/_downloads/7191118aef248d2adc5fe59048afe782/h2.xyz
from google.colab import files
uploaded = files.upload()

In [ ]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Building the electronic Hamiltonian

In [8]:
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [9]:
h, qubits = qchem.molecular_hamiltonian(
    name,
    geometry,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=2,
    active_orbitals=2,
    mapping='jordan_wigner'
)

print('Number of qubits = ', qubits)
print('Hamiltonian is ', h)

Number of qubits =  4
Hamiltonian is  (-0.04207897647782276) [I0]
+ (0.17771287465139946) [Z0]
+ (0.1777128746513994) [Z1]
+ (-0.24274280513140462) [Z2]
+ (-0.24274280513140462) [Z3]
+ (0.17059738328801052) [Z0 Z1]
+ (0.04475014401535161) [Y0 X1 X2 Y3]
+ (-0.04475014401535161) [Y0 Y1 X2 X3]
+ (-0.04475014401535161) [X0 X1 Y2 Y3]
+ (0.04475014401535161) [X0 Y1 Y2 X3]
+ (0.12293305056183798) [Z0 Z2]
+ (0.1676831945771896) [Z0 Z3]
+ (0.1676831945771896) [Z1 Z2]
+ (0.12293305056183798) [Z1 Z3]
+ (0.17627640804319591) [Z2 Z3]


In [ ]:
# Implementing the VQE algorithm

In [10]:
dev = qml.device('default.qubit', wires=qubits)

def circuit(params, wires):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])
    # why setting the CNOT gates in this way?

In [11]:
# Cost function
cost_fn = qml.ExpvalCost(circuit, h, dev)

In [12]:
# Optimization
opt = qml.GradientDescentOptimizer(stepsize=0.4)
np.random.seed(0)
params = np.random.normal(0, np.pi, (qubits, 3))

print(params)

[[ 5.54193389  1.25713095  3.07479606]
 [ 7.03997361  5.86710646 -3.07020901]
 [ 2.98479079 -0.47550269 -0.32427159]
 [ 1.28993324  0.45252622  4.56873497]]


In [13]:
max_iterations = 200
conv_tol = 1e-06

prev_energy = cost_fn(params)
for n in range(max_iterations):
    params = opt.step(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print('Iteration = {:},  Energy = {:.8f} Ha'.format(n, energy))

    if conv <= conv_tol:
        break

    prev_energy = energy

print()
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('Final value of the ground-state energy = {:.8f} Ha'.format(energy))
print('Accuracy with respect to the FCI energy: {:.8f} Ha ({:.8f} kcal/mol)'.format(
    np.abs(energy - (-1.136189454088)), np.abs(energy - (-1.136189454088))*627.503
    )
)
print()
print('Final circuit parameters = \n', params)

Iteration = 0,  Energy = -0.88179557 Ha
Iteration = 20,  Energy = -1.13380513 Ha
Iteration = 40,  Energy = -1.13558756 Ha
Iteration = 60,  Energy = -1.13585794 Ha
Iteration = 80,  Energy = -1.13600617 Ha
Iteration = 100,  Energy = -1.13608848 Ha
Iteration = 120,  Energy = -1.13613394 Ha

Final convergence parameter = 0.00000099 Ha
Final value of the ground-state energy = -1.13615709 Ha
Accuracy with respect to the FCI energy: 0.00003237 Ha (0.02031093 kcal/mol)

Final circuit parameters = 
 [[ 5.54193389e+00  1.30219523e-08  3.07479606e+00]
 [ 7.03997361e+00  6.28318530e+00 -3.07020901e+00]
 [ 2.98479079e+00 -2.09540998e-01 -4.16893297e-02]
 [ 1.28993324e+00  1.30913909e-12  4.56873497e+00]]


In [ ]:
# test another circuit with different CNOT arrangements

In [20]:
dev = qml.device('default.qubit', wires=qubits)

def circuitTest(params, wires):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    # setting the CNOT gates in a different way

In [21]:
# Cost function
cost_Test = qml.ExpvalCost(circuitTest, h, dev)

In [22]:
# Optimization
opt = qml.GradientDescentOptimizer(stepsize=0.4)
np.random.seed(0)
params = np.random.normal(0, np.pi, (qubits, 3))

print(params)

[[ 5.54193389  1.25713095  3.07479606]
 [ 7.03997361  5.86710646 -3.07020901]
 [ 2.98479079 -0.47550269 -0.32427159]
 [ 1.28993324  0.45252622  4.56873497]]


In [23]:
max_iterations = 200
conv_tol = 1e-06

prev_energy = cost_Test(params)
for n in range(max_iterations):
    params = opt.step(cost_Test, params)
    energy = cost_Test(params)
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print('Iteration = {:},  Energy = {:.8f} Ha'.format(n, energy))

    if conv <= conv_tol:
        break

    prev_energy = energy

print()
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('Final value of the ground-state energy = {:.8f} Ha'.format(energy))
print('Accuracy with respect to the FCI energy: {:.8f} Ha ({:.8f} kcal/mol)'.format(
    np.abs(energy - (-1.136189454088)), np.abs(energy - (-1.136189454088))*627.503
    )
)
print()
print('Final circuit parameters = \n', params)

Iteration = 0,  Energy = -0.21400133 Ha
Iteration = 20,  Energy = -0.51583110 Ha
Iteration = 40,  Energy = -0.52089852 Ha
Iteration = 60,  Energy = -0.52172470 Ha
Iteration = 80,  Energy = -0.52185948 Ha

Final convergence parameter = 0.00000091 Ha
Final value of the ground-state energy = -0.52187598 Ha
Accuracy with respect to the FCI energy: 0.61431348 Ha (385.48355048 kcal/mol)

Final circuit parameters = 
 [[ 5.54193389e+00  2.87321832e-05  3.07687141e+00]
 [ 7.03997361e+00  6.28318531e+00 -3.07020901e+00]
 [ 2.98479079e+00 -3.27071402e-09 -3.24271587e-01]
 [ 1.28993324e+00  1.31296649e-02  4.56873497e+00]]


In [ ]:
# Note that the result is not as good as the original circuit
# It is crucial to set a proper circuit! 